In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
Init();


In [2]:
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 0; Grid Count = 0; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D }

In [3]:
BoSSSshell.GetDefaultQueue()

RuntimeLocation,win\amd64
AdditionalEnvironmentVars,"[ [OMP_PROC_BIND, spread] ]"
DeploymentBaseDirectory,\\dc3\userspace\sebastian\cluster\binaries
DeployRuntime,True
Name,Default
DotnetRuntime,dotnet
Username,FDY\sebastian
NumOfAdditionalServiceCores,0
NumOfAdditionalServiceCoresMPISerial,0
NumOfServiceCoresPerMPIprocess,0
ServerName,DC3


In [4]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D

In [5]:
int[] Configs = new int[] {1,2,3};
int[] DGdegrees = new int[] {3};
int[] Grid_Res = new int[] {1};
double[] Amplitudes = new double[] {1e-5};
string[] WaveForms= new string[] {"1sinus"};
string[] LevelSetConfigs = new string[] {"true"};//,"false"};
//double[] MachLs = GenericBlas.Linspace(2.0,8.0,7);
double[] MachLs = new double[] {1.5};
double[] WLs = new double[] {0.8};

In [6]:
//foreach(double iProb in iProbs) {
    double amp=1e-5;
    double amp_neg=0.0;
    double amp_pos=amp;
    double endTime=1.5;
    double shockPos=1.5;
    double wavePos=0.0;
    double scaling=1;
    double waveLength=0.4;
    double MachL=1.50;
                foreach(double waveLength in WLs ){
            foreach(double MachL in MachLs ){
        foreach(double amp in Amplitudes ){
foreach(double config in Configs) {
foreach(string waveform in WaveForms) {
    foreach(string wLS in LevelSetConfigs){
        amp_neg= 0.0;            amp_pos= amp;
         if(config==1){ //downstream negative acoustic wave (slow)
            amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;          shockPos=0.5;            wavePos=0.9;
        }else if(config==2){ //upstream positiv acoustic wave (fast)
            amp_neg= 0.0;            amp_pos= amp;            endTime=1.5;            shockPos=1.5;            wavePos=0.0;
        }else if(config==3){ //upstream negative acoustic wave (still fast)
            amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;           shockPos=1.5;            wavePos=0.3;
        }
    
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
                //int cells=(n+1)*30;
                int cells =(n+1)*30+1;
                Job j = new Job(string.Format($"AcousticWave1D_p{p}_xCells{cells}_tCells{cells}_agg{0.4}_wLS{wLS}_{waveform}_Mach{MachL}_wP{wavePos}_sP{shockPos}_amppos{amp_pos}_amp_neg{amp_neg}_scaling{scaling}_wL{waveLength}_rc2"), typeof(XESTSF.XESTSFMain));// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                j.RetryCount = 2;
                j.MySetCommandLineArguments("--control", $"cs:XESTSF.XESTSFHardCodedControl.AcousticWave1D(waveLength:{waveLength},scaling:{scaling},endTime:{endTime},MachL:{MachL},withLevelSet:{wLS},wavePosition:{wavePos}, shockPosition:{shockPos},p_amp_neg: {amp_neg}, p_amp_pos:{amp_pos},waveform:\"{waveform}\",dgDegree:{p},numOfCellsX:{cells},PlotInterval:5,numOfCellsT:{cells},dbPath:@\"{dbPath}\")");
                j.NumberOfThreads=1;
                j.Activate(BoSSSshell.GetDefaultQueue()); 
                
            
        };                                                                                                                                                          
    };
};
    
}
}
        }
            }
                }

